In [1]:
import iris
import numpy
import matplotlib.pyplot as plt
import glob

import pandas as pd
#from bokeh.charts import TimeSeries, show, output_file

from iris.experimental.equalise_cubes import equalise_attributes

## Read data

In [2]:
# Configuration

model = 'NorESM1-M'
control_location = '/g/data/ua6/DRSv3/CMIP5'  # /g/data/r87/dbi599
historical_location = '/g/data/ua6/DRSv3/CMIP5'

depth = 15
latitude = 56
longitude = 30

In [ ]:
control_files = glob.glob('%s/piControl/mon/ocean/thetao/r1i1p1/thetao_Omon_%s_piControl_r1i1p1_*.nc' %(control_location, institution, model, model) )
historical_files = glob.glob('%s/historical/mon/ocean/thetao/r1i1p1/thetao_Omon_%s_historical_r1i1p1_*.nc' %(historical_location, institution, model, model) )
dedrifted_historical_files = glob.glob('%s/historical/mon/ocean/thetao/r1i1p1/dedrifted/thetao_Omon_%s_historical_r1i1p1_*.nc' %(institution, model, model) )
coefficient_file = '/g/data/r87/dbi599/DRSv2/CMIP5/%s/piControl/mon/ocean/thetao/r1i1p1/thetao-coefficients_Omon_%s_piControl_r1i1p1_all.nc' %(institution, model, model)

In [ ]:
lev_constraint = iris.Constraint(depth=depth)
lat_constraint = iris.Constraint(grid_latitude=latitude)
lon_constraint = iris.Constraint(grid_longitude=longitude)

In [ ]:
def read_files(file_list):
    """Read a list of files into an iris Cube"""
    
    cube = iris.load(file_list, 'sea_water_potential_temperature' & lev_constraint & lat_constraint & lon_constraint)
    equalise_attributes(cube)
    cube = cube.concatenate_cube()
    
    return cube

In [ ]:
control_cube = read_files(control_files)
historical_cube = read_files(historical_files)
dedrifted_historical_cube = read_files(dedrifted_historical_files)
coefficient_cube = iris.load_cube(coefficient_file, 'Sea Water Potential Temperature' & lev_constraint & lat_constraint & lon_constraint)

In [ ]:
def apply_polynomial(x, a, b, c, d):
    """Evaluate cubic polynomial.

    The axis argument is not used but is required for the function to be 
      used with numpy.apply_over_axes 

    """

    result = a + b * x + c * x**2 + d * x**3 
    
    return result

In [ ]:
coeff_a, coeff_b, coeff_c, coeff_d = coefficient_cube.data
my_poly = apply_polynomial(control_cube.coord('time').points,
                           coeff_a, coeff_b, coeff_c, coeff_d)

In [ ]:
numpy_poly = numpy.poly1d([float(coeff_d), 
                           float(coeff_c),
                           float(coeff_b),
                           float(coeff_a)])

## Plot

In [ ]:
control_times = control_cube.coord('time').points
df = pd.DataFrame(control_cube.data, index=control_times, columns=['control'])

In [ ]:
plt.plot(full_control_cube.coord('time').points[0:50], my_poly[0:50], 'o')
plt.plot(control_cube.coord('time').points[0:50], control_cube.data[0:50])
plt.ylabel('y')
plt.xlabel('x')
plt.show()